Clase 5
====
Hoy veremos herramientas avanzadas de cobrapy.

Objetivos

* Lectura de modelos metabólicos
* Flux Varional Analysis (FVA)
* Parsimonious FBA


Lectura de modelos metabólicos
===========================

Cobrapy puede leer y escribir modelos metabólicos en varios formatos, incluyendo SBML, JSON, YAML, MAT, y pickle. Hoy trabajaremos con SBML, el cual es el formato preferido. 

Los modelos metabólicos para varias especies pueden ser descargados [aquí](http://systemsbiology.ucsd.edu/Downloads). Para los ejercicios de hoy usaremos la red de *Escherichia coli* que viene dentro de los ejemplos de cobra.


In [1]:
# Importar cobra y los casos de prueba
import cobra
import cobra.test
# Cargar el modelo de E coli
model = cobra.test.create_test_model("ecoli")

In [3]:
# Número de reacciones, metabolitos, y función objetivo
print( len(model.reactions) )
print( len(model.metabolites) )
print( model.objective )

2583
1805
Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1


In [4]:
# Simulación de la distribución de fluxes usando FBA
solution = model.optimize()
# Resumen de los fluxes de sustratos, productos, y reacción de biomasa
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.005113,0,0.00%
cl_e,EX_cl_e,0.005113,0,0.00%
cobalt2_e,EX_cobalt2_e,2.456E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006965,0,0.00%
fe2_e,EX_fe2_e,0.01578,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1918,0,0.00%
mg2_e,EX_mg2_e,0.008522,0,0.00%
mn2_e,EX_mn2_e,0.0006788,0,0.00%
mobd_e,EX_mobd_e,0.0001267,0,0.00%


In [5]:
# Reacciones en las que participa el sustrato limitante: glucosa
for reaction in model.metabolites.get_by_id('glc__D_e').reactions:
    print( reaction.id, reaction.bounds )

EX_glc__D_e (-10.0, 1000.0)
GLCtex_copy1 (-1000.0, 1000.0)
GLCtex_copy2 (0.0, 1000.0)


In [7]:
# Recalculo del flux de biomasa, esta vez restringiendo aún más el consumo de glucosa
model.reactions.get_by_id("EX_glc__D_e").bounds=(-5,1000)
solution = model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.002523,0,0.00%
cl_e,EX_cl_e,0.002523,0,0.00%
cobalt2_e,EX_cobalt2_e,1.212E-05,0,0.00%
cu2_e,EX_cu2_e,0.0003436,0,0.00%
fe2_e,EX_fe2_e,0.007784,0,0.00%
glc__D_e,EX_glc__D_e,5,6,100.00%
k_e,EX_k_e,0.0946,0,0.00%
mg2_e,EX_mg2_e,0.004204,0,0.00%
mn2_e,EX_mn2_e,0.0003349,0,0.00%
mobd_e,EX_mobd_e,6.252E-05,0,0.00%


Running Flux Variational Analysis (FVA)
=====================================

En general FBA no tiene una solución única, pero varias que resultan en el mismo valor de la función objetivo. El análisis de variabilidad de flujo (FVA: Flux Variational Analysis) permite encontra rel rango de flujos de cada metabólito en el valor optimo.

In [8]:
# Primero encontramos la solución óptima del modelo
solution = model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.002523,0,0.00%
cl_e,EX_cl_e,0.002523,0,0.00%
cobalt2_e,EX_cobalt2_e,1.212E-05,0,0.00%
cu2_e,EX_cu2_e,0.0003436,0,0.00%
fe2_e,EX_fe2_e,0.007784,0,0.00%
glc__D_e,EX_glc__D_e,5,6,100.00%
k_e,EX_k_e,0.0946,0,0.00%
mg2_e,EX_mg2_e,0.004204,0,0.00%
mn2_e,EX_mn2_e,0.0003349,0,0.00%
mobd_e,EX_mobd_e,6.252E-05,0,0.00%


In [9]:
# Luego computamos los rangos de flujo de cada metabólito
from cobra.flux_analysis import flux_variability_analysis
fva=flux_variability_analysis(model)

In [8]:
# Para ver los rangos de los primeros 10 metabolitos
#print fva[0:10]
print model.metabolites.get_by_id('glc__D_e').reactions
print "value"
print fva.loc['EX_glc_e']

frozenset([<Reaction EX_glc_e at 0x7fae6ae3ae90>, <Reaction GLCtex at 0x7fae6a5459d0>, <Reaction GLCtexi at 0x7fae6a545d90>])
value
minimum   -5.0
maximum   -5.0
Name: EX_glc_e, dtype: float64


In [9]:
# De manera más concisa, considerando hasta un valor de biomasa de un 5% (0.95) menos del óptimo
solution = model.optimize()
model.summary(fva=1)

IN FLUXES                                 OUT FLUXES                   OBJECTIVES
----------------------------------------  ---------------------------  ----------------------
id            Flux  Range                 id       Flux  Range         Ec_biomass_i...  0.485
--------  --------  --------------------  -----  ------  ------------
o2_e      9.07      [9.07, 259]           h2o_e   22.9   [22.9, 523]
nh4_e     5.23      [5.23, 5.23]          co2_e   10.1   [10.1, 10.1]
glc__D_e  5         [5, 5]                h_e      4.45  [4.45, -996]
pi_e      0.468     [0.468, 0.468]        fe3_e    0     [0, 1e+03]
so4_e     0.122     [0.122, 0.122]
k_e       0.0946    [0.0946, 0.0946]
fe2_e     0.00778   [0.00778, 1e+03]
mg2_e     0.0042    [0.0042, 0.0042]
ca2_e     0.00252   [0.00252, 0.00252]
cl_e      0.00252   [0.00252, 0.00252]
cu2_e     0.000344  [0.000344, 0.000344]
mn2_e     0.000335  [0.000335, 0.000335]
zn2_e     0.000165  [0.000165, 0.000165]
ni2_e     0.000157  [0.000157, 0.000

In [10]:
# Esta vez considerando que biomasa pueda bajar hasta en un 5% (0.95) de su valor óptimo
solution = model.optimize()
model.summary(fva=1)

IN FLUXES                                 OUT FLUXES                   OBJECTIVES
----------------------------------------  ---------------------------  ----------------------
id            Flux  Range                 id       Flux  Range         Ec_biomass_i...  0.485
--------  --------  --------------------  -----  ------  ------------
o2_e      9.07      [9.07, 259]           h2o_e   22.9   [22.9, 523]
nh4_e     5.23      [5.23, 5.23]          co2_e   10.1   [10.1, 10.1]
glc__D_e  5         [5, 5]                h_e      4.45  [4.45, -996]
pi_e      0.468     [0.468, 0.468]        fe3_e    0     [0, 1e+03]
so4_e     0.122     [0.122, 0.122]
k_e       0.0946    [0.0946, 0.0946]
fe2_e     0.00778   [0.00778, 1e+03]
mg2_e     0.0042    [0.0042, 0.0042]
ca2_e     0.00252   [0.00252, 0.00252]
cl_e      0.00252   [0.00252, 0.00252]
cu2_e     0.000344  [0.000344, 0.000344]
mn2_e     0.000335  [0.000335, 0.000335]
zn2_e     0.000165  [0.000165, 0.000165]
ni2_e     0.000157  [0.000157, 0.000

parsimonious FBA
=========

Por otra parte el valor optimo encontrado por FBA no necesariamente implica que se están ocupando el minimo de flujos disponibles. Por ejemplo, cuando hay ciclos es posible que algunos flujos tomen valores arbitrariamente altos sin afectar el valor de la función objetivo. Esto puede ser una causa de artefactos. Para prevenir esto se puede usar la herramienta parsimoniouos FBA (pFBA), la cual realiza una optimización en dos niveles. En el nivel interna se optimiza la función objetivo, tal como en un FBA normal, mientras que en la optimización externa se minimiza la suma total de los flujos con la restricción de que se conserve el valor óptimo encontrado por la optimización interna.

Veamos un ejemplo:

In [11]:
# Este es un FBA regular
fba_solution = model.optimize()
# Ahora veamos como seria el pFBA
pfba_solution = cobra.flux_analysis.pfba(model)

In [14]:
# Los valores de las funciones objetivos son:
print( fba_solution.objective_value)
print( pfba_solution.objective_value)

0.48465335156068157
355.24974817497935


In [15]:
# Los valores de la reaccion de biomasa (funcion objetivo del FBA) deberia ser el mismo en ambos casos
print( fba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"])
print( pfba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"])
print( abs(fba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"] - pfba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]))

0.48465335156068157
0.4846533515606814
1.6653345369377348e-16


In [17]:
# Sin embargo la suma de los flujos deberia ser menor en pFBA
print( sum(fba_solution.fluxes))
print( sum(pfba_solution.fluxes))

108.9412421656483
112.41919386979679


Ejercicio
=========

1. Calculen el valor de los flujos usando geomtric FBA, una alternavita a pFBA que encuentra una solución única [(aquí las instrucciones)](https://cobrapy.readthedocs.io/en/latest/simulating.html#Running-geometric-FBA).
2. Escriban una función para realizar todos los análsis vistos en este clase